<a href="https://colab.research.google.com/github/Tena-rin/Kintsugi-Splitter/blob/main/Scripts/step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# step2: Kintsugi-line extraction

## これは何？
このノートは、金継ぎ線の抽出を行うための実装です。

## できること
- 1. 入力画像(input.png)を読み込む
- 2. 画像に含まれない色(best_rgb)を検出
- 3. 金継ぎ線っぽい領域を抽出する
- 4. マスク画像(output.png)として保存する

## 実行方法
上から順番にセルを実行してください。


In [ ]:
!pip -q install google-genai
import cv2
import os
import numpy as np
from PIL import Image
from google import genai
from google.genai import types

In [ ]:
def kmeans_colors_bgr(img_bgr, k=12, max_samples=200_000, seed=0):
    pixels = img_bgr.reshape(-1, 3).astype(np.float32)

    if len(pixels) > max_samples:
        rng = np.random.default_rng(seed)
        idx = rng.choice(len(pixels), size=max_samples, replace=False)
        pixels = pixels[idx]

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1.0)
    flags = cv2.KMEANS_PP_CENTERS
    cv2.setRNGSeed(seed)

    _, _, centers = cv2.kmeans(pixels, k, None, criteria, 3, flags)
    return centers.astype(np.uint8)


def min_dist_rgb_to_centers(rgb, centers_rgb):
    diff = centers_rgb.astype(np.float32) - np.array(rgb, dtype=np.float32)
    dist = np.linalg.norm(diff, axis=1)
    return float(dist.min())


def detect_absent_colors(
    image_path,
    palette_rgb,
    k=12,
    threshold=35,
    resize_max=800
):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(image_path)

    h, w = img.shape[:2]
    scale = min(resize_max / max(h, w), 1.0)
    if scale < 1.0:
        img = cv2.resize(img, (int(w * scale), int(h * scale)))

    centers_bgr = kmeans_colors_bgr(img, k=k)
    centers_rgb = centers_bgr[:, ::-1]

    present, absent = [], []
    for rgb in palette_rgb:
        d = min_dist_rgb_to_centers(rgb, centers_rgb)
        if d <= threshold:
            present.append((rgb, d))
        else:
            absent.append((rgb, d))

    return present, absent, centers_rgb


def pick_best_absent_color(absent_list):
    if not absent_list:
        return None
    return max(absent_list, key=lambda x: x[1])

def rgb_to_text(rgb):
    r, g, b = rgb
    return f"RGB({r},{g},{b})"


def build_prompt(best_rgb):
    color_text = rgb_to_text(best_rgb)

    return f"""
この画像にはどのようなものが写されているかを、具体的に説明してください。

その上で、この画像を直接編集してください。
編集内容は以下の通りです。

- 割れ目に沿った金継ぎの金色の線の部分のみを、
  {color_text} の単色で、
  むらなく、立体感や陰影が残らないように塗りつぶしてください。
- 皿の表面や背景部分には一切色を塗らないでください。
- 塗られた部分は完全にフラットな色になるようにしてください。

編集後の画像を出力してください。
""".strip()

if __name__ == "__main__":

    IMAGE_PATH = "input.png"

    PALETTE = [
        (12, 238, 69),     # green
        (255, 255, 0),     # yellow
        (0, 255, 255),     # cyan
        (255, 0, 255),     # magenta
        (0, 0, 0),         # black
        (255, 255, 255),   # white
        (128, 128, 128),   # gray
    ]

    present, absent, centers_rgb = detect_absent_colors(
        IMAGE_PATH,
        PALETTE,
        k=12,
        threshold=35
    )

    print("=== IMAGE CENTERS ===")
    for c in centers_rgb:
        print(f"RGB({c[0]},{c[1]},{c[2]})")

    print("\n=== ABSENT COLORS ===")
    for (r, g, b), d in absent:
        print(f"RGB({r},{g},{b})  dist={d:.2f}")

    best_rgb, best_dist = pick_best_absent_color(absent)
    print(f"\nBEST COLOR: RGB{best_rgb}  dist={best_dist:.2f}")

    # プロンプト生成
    prompt = build_prompt(best_rgb)
    print("\n=== PROMPT ===")
    print(prompt)

    client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

    response = client.models.generate_content(
        model="gemini-3-pro-image-preview",
        contents=[
            prompt,
            Image.open(IMAGE_PATH),
        ],
        config=types.GenerateContentConfig(
            response_modalities=["TEXT", "IMAGE"],
        )
    )

    for part in response.parts:
        if part.text is not None:
            print("\n=== MODEL TEXT OUTPUT ===")
            print(part.text)
        elif image := part.as_image():
            image.save("output.png")
            print("output.png")
